<a href="https://colab.research.google.com/github/vrathi101/SIGCOMMInclusivity/blob/main/Connect_ManuallyCuratedData_with_SIGCOMMData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install and import libraries which will be used
!pip install fuzzywuzzy
!pip install levenshtein
from fuzzywuzzy import fuzz
import pandas as pd
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Reading and displaying the PublisherNames csv which was written as a csv in the last Google Colab.

In [3]:
proj_path = '/content/gdrive/MyDrive/Colab Notebooks/save_data/people_weave/sigcomm_members/'
dfSigcomm = pd.read_csv(proj_path + 'SIGCOMMNames.csv')

In [4]:
# display some data characteristics
for column_name, column_type in zip(dfSigcomm.columns, dfSigcomm.dtypes):
    print(f"Column Name: {column_name}, Data Type: {column_type}")
print(len(dfSigcomm))
print(dfSigcomm.head(5))

Column Name: name, Data Type: object
Column Name: year, Data Type: int64
Column Name: committee, Data Type: object
Column Name: region, Data Type: object
Column Name: cleanedName, Data Type: object
Column Name: cleanedRegion, Data Type: object
Column Name: Continent, Data Type: object
1230
                        name  year   committee                      region  \
0            Pablo Rodriguez  2009  Organizing  Telefonica Research, Spain   
1             Ernst Biersack  2009  Organizing             Eurecom, France   
2  Konstantina  Papagiannaki  2009  Organizing  Intel Labs Pittsburgh, USA   
3                Luigi Rizzo  2009  Organizing   Universit� di Pisa, Italy   
4            Christophe Diot  2009  Organizing             Thomson, France   

                 cleanedName               cleanedRegion Continent  
0            Pablo Rodriguez  Telefonica Research, Spain        EU  
1             Ernst Biersack             Eurecom, France        EU  
2  Konstantina  Papagiannaki  Int

Uploading the manually-curated researchers data CSV into the Colab and adjusting as needed, including removing duplicates and selecting desired columns.
Imported the Researcher CSV with manually curated researcher data for many committees, including SIGCOMM. Then made adjustments as needed icluding removing duplicates and selecting desired columns.

In [6]:
# Function to decode the string with fallback encoding
def decode_with_fallback(text, primary_encoding, fallback_encoding):
    try:
        if isinstance(text, str):
            return text.encode(primary_encoding).decode(primary_encoding)
        else:
            return str(text).encode(fallback_encoding).decode(fallback_encoding)
    except UnicodeDecodeError:
        return str(text).encode(fallback_encoding).decode(fallback_encoding)


dfResearchers = pd.read_csv(proj_path + 'researcher.csv')
# Specify the columns to extract from the 'researchers' DataFrame
columns_to_extract =['Name','Female or NB', 'Region', 'Approximate Doctoral Degree Graduation Year']
# Create a new DataFrame 'dfResearcher' with the selected columns
dfResearcher = dfResearchers[columns_to_extract]
dfResearcher.rename(columns={'Female or NB': 'Gender'}, inplace=True)
dfResearcher.rename(columns={'Approximate Doctoral Degree Graduation Year': 'ApproximateDoctoralGradYear'}, inplace=True)
dfResearcher['Gender'] = dfResearcher['Gender'].replace({float('Nan'): 'Male', 1.0: 'Female'})
dfResearcher['Region'].fillna('NorAm', inplace=True)
print(len(dfResearcher))
print(dfResearcher.groupby(['Gender'])['Gender'].count())
# Apply the decoding function to create the 'cleanedName' column
dfResearcher['cleanedName'] = dfResearcher['Name'].apply(lambda x: decode_with_fallback(x, 'utf-8', 'unicode_escape'))
dfResearcher.drop_duplicates(subset = 'cleanedName', inplace=True)
print(dfResearcher.head())

1033
Gender
Female    162
Male      871
Name: Gender, dtype: int64
              Name  Gender Region ApproximateDoctoralGradYear      cleanedName
0  Sergey Gorinsky    Male     EU                        2003  Sergey Gorinsky
1   Hitesh Ballani    Male     EU                        2009   Hitesh Ballani
2     Haya Shulman  Female     EU                        2014     Haya Shulman
3  Rüdiger Birkner    Male     EU                        2021  Rüdiger Birkner
4     Klaus Wehrle    Male     EU                        2002     Klaus Wehrle


<ipython-input-6-0007f6083727>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfResearcher.rename(columns={'Female or NB': 'Gender'}, inplace=True)
<ipython-input-6-0007f6083727>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfResearcher.rename(columns={'Approximate Doctoral Degree Graduation Year': 'ApproximateDoctoralGradYear'}, inplace=True)
<ipython-input-6-0007f6083727>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

Adding columns to dfSigcomm (original) dataframe by comparing it with dfResearcher dataframe based on the full name of a committee member and a fuzzy match. Another column is used for the gender-guesser library which guesses the gender for every SIGCOMM member in dfSigcomm.
Additionally, for all the committee members whose continent of location could not be determined, it will check if the dfResearcher has these names with their regions and add it in accordingly.

In [7]:
#add a DirectMatch column which puts in gender if found in dfResearcher based on code
dfSigcomm['DirectMatch'] = ''
for index, row in dfSigcomm.iterrows():
  chkName=row['cleanedName']
  gender=''
  if (chkName in dfResearcher['cleanedName'].values):
    first_location  = dfResearcher.loc[dfResearcher['cleanedName'] == chkName].index[0]
    gender = dfResearcher.loc[first_location, 'Gender']
    dfSigcomm.at[index, 'DirectMatch'] = gender
print(dfSigcomm.groupby(['DirectMatch'])['DirectMatch'].count())

#add a FuzzyMatch column which puts in gender if found in dfResearcher based on code
dfSigcomm['FuzzyMatch'] = ''
dfSigcomm['ApproximateDoctoralGradYear'] = ''
for idx, row in dfSigcomm.iterrows():
    name_updated = row['cleanedName']
    region_updated = row['cleanedRegion']
    continent = row['Continent']
    gradYear = row['ApproximateDoctoralGradYear']
    best_match = None
    best_score = 0
    researcher_row = None
    for _, research_row in dfResearcher.iterrows():
        name_research = research_row['cleanedName']
        score = fuzz.token_set_ratio(name_updated, name_research)
        if score > best_score and score > 90:
            best_match = research_row['Gender']
            best_score = score
            researcher_row = research_row
        if score == 100:  # If perfect match, break the loop
            break
    dfSigcomm.loc[idx, 'FuzzyMatch'] = best_match
    if pd.isna(continent) and researcher_row is not None:
      dfSigcomm.loc[idx, 'Continent'] = researcher_row['Region']
    if gradYear.strip()=='' and researcher_row is not None:
      dfSigcomm.loc[idx, 'ApproximateDoctoralGradYear'] = researcher_row['ApproximateDoctoralGradYear']
print(dfSigcomm.groupby(['FuzzyMatch'])['FuzzyMatch'].count())

DirectMatch
          168
Female    231
Male      831
Name: DirectMatch, dtype: int64
FuzzyMatch
Female    262
Male      964
Name: FuzzyMatch, dtype: int64


Changing the names of the values in the 'Continent' column to their full names. This will be used in the graphs in the next Google Colab.

In [8]:
continent_mapping = {
    'NorAm': 'North America',
    'AS': 'Asia',
    'Asia': 'Asia',
    'OC': 'Australia',
    'Australia': 'Australia',
    'SA': 'South America',
    'Africa': 'Africa',
    'AF': 'Africa',
    'EU': 'Europe'
}
dfSigcomm['Continent'] = dfSigcomm['Continent'].replace(continent_mapping)

Making final adjustments to the dfSigcomm dataframe to prepare it for data visualization.

In [9]:
name_pairs = [
    ('Jitendra Padhye', 'Jitu Padhye'),
    ('Jaorg Ott', 'Joerg Ott'),
    ('Zhi-Li Zhang', 'Zhili Zhang')
]
for name_a, name_b in name_pairs:
    rows_a = dfSigcomm[dfSigcomm['cleanedName'] == name_a]
    rows_b = dfSigcomm[dfSigcomm['cleanedName'] == name_b]
    data_copy = rows_a.drop(columns=['year', 'committee', 'name', 'cleanedName']).iloc[0]

    for index, row in rows_b.iterrows():
        for column in data_copy.index:
            if column not in ['year', 'committee', 'name', 'cleanedName']:
                value = data_copy[column]
                dfSigcomm.at[index, column] = value
print(dfSigcomm)

                           name  year   committee  \
0               Pablo Rodriguez  2009  Organizing   
1                Ernst Biersack  2009  Organizing   
2     Konstantina  Papagiannaki  2009  Organizing   
3                   Luigi Rizzo  2009  Organizing   
4               Christophe Diot  2009  Organizing   
...                         ...   ...         ...   
1225                 Yiting Xia  2023     Program   
1226               Yiying Zhang  2023     Program   
1227                     Yu Hua  2023     Program   
1228                 Yunxin Liu  2023     Program   
1229              Zhizhen Zhong  2023     Program   

                                                 region  \
0                            Telefonica Research, Spain   
1                                       Eurecom, France   
2                            Intel Labs Pittsburgh, USA   
3                             Universit� di Pisa, Italy   
4                                       Thomson, France   
...      

We write the current dataframe into a CSV file which will be read and analyzed in the next Google Colab.

In [10]:
dfSigcomm.to_csv("/content/gdrive/My Drive/Colab Notebooks/save_data/people_weave/sigcomm_members/SIGCOMMNamesAdjusted.csv", sep=',', index=False, encoding='utf-8')